In [1]:
%load_ext tensorboard

# load tensorflow dependencies
import tensorflow as tf
from classification_models.tfkeras import Classifiers
from tensorflow.keras.utils import to_categorical
from tensorflow_addons.metrics import CohenKappa
from tensorflow_addons.layers import GroupNormalization
from tensorflow.keras import layers as KL
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
import tensorflow.keras.backend as K
import efficientnet.tfkeras as efn

# 16 bit precision computing
from tensorflow.keras.mixed_precision import experimental as mixed_precision
print(tf.__version__)
print(tf.config.experimental.list_physical_devices())


from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path
from pprint import pprint
import pandas as pd
from glob import glob
import skimage.io
import sys

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import cohen_kappa_score, make_scorer
import time


sys.path.insert(0,'..')
from model.layers import GeneralizedMeanPooling2D

# custom packages
from preprocessing.utils.data_loader import PandasDataLoader
from preprocessing.generators import TiffGenerator, TiffFromCoords
from utils.utils import set_gpu_memory, seed_all

from model.network import Network

# Augmentation packages

# custom stain augmentation
from preprocessing.augmentations import StainAugment

import albumentations as A
from albumentations import (
    Flip, ShiftScaleRotate, RandomRotate90,
    ShiftScaleRotate, Blur, OpticalDistortion, RandomBrightnessContrast, 
    OneOf, Compose, RandomScale, ElasticTransform, GaussNoise, GaussianBlur,
    RandomBrightness, RandomContrast
)

import efficientnet_gn.tfkeras as efn2

2.2.0
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'), PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'), PhysicalDevice(name='/physical_device:XLA_GPU:1', device_type='XLA_GPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [2]:
seed_all(20)
DATA_DIR = Path('../data/')

# control for the gpu memory, and number of used gpu's
set_gpu_memory(device_type='GPU')
# see the utils functions which seeds are set
# tensorflow ops still have to be seeded manually...
seed_all()

policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)

print('Compute dtype: %s' % policy.compute_dtype)
print('Variable dtype: %s' % policy.variable_dtype)

2 Physical GPUs, 1 Logical GPUs
Compute dtype: float16
Variable dtype: float32


In [23]:
bottleneck = efn2.EfficientNetB1( 
    include_top=False, 
    pooling='avg',
    weights=None
)


In [24]:
bottleneck.get_weights()

[array([[[[-1.06096834e-01, -1.29892632e-01,  3.18787131e-03,
            8.98062885e-02, -4.31370595e-03, -2.29268800e-02,
           -1.75840542e-01, -7.88071007e-03,  9.77827907e-02,
            1.10137269e-01,  1.50763197e-02, -7.64639676e-02,
           -1.77310202e-02,  7.28444979e-02,  7.72056207e-02,
           -7.74862757e-03, -1.07580692e-01, -2.35067308e-02,
           -8.43559280e-02, -4.36140262e-02,  2.46360321e-02,
           -1.72423553e-02,  8.74456912e-02, -2.76545938e-02,
            1.33094236e-01, -3.06873303e-02,  4.25696000e-02,
           -8.65394175e-02,  1.14161044e-01, -3.84231168e-03,
            6.61009699e-02, -8.28354247e-03],
          [ 8.95384103e-02,  4.18353304e-02, -1.63637817e-01,
           -1.69470504e-01,  4.68403213e-02,  3.50762270e-02,
            1.11427670e-03, -5.03748506e-02, -1.21745043e-01,
            3.42609771e-02, -1.64239883e-01,  1.55808568e-01,
            4.44159620e-02, -5.73206060e-02, -5.17727733e-02,
           -3.62416692e-

In [29]:
bottleneck.load_weights('../efficientnet_weights/efficientnetb0.h5', by_name=True)

In [30]:
bottleneck.get_weights()

[array([[[[-6.31598294e-01,  1.58087373e-01,  3.36237028e-02,
           -4.10183929e-02, -1.03312828e-01, -1.30804516e-02,
            0.00000000e+00, -9.79949161e-02, -1.20587930e-01,
            8.35960209e-02, -1.30694389e-01, -8.63404572e-02,
            1.28677368e-01,  5.17749727e-01,  4.05013412e-01,
           -1.95988178e-01,  1.57250032e-01,  6.61734521e-01,
            1.62455797e-01,  7.02280551e-04,  4.73497994e-03,
           -1.48393601e-01,  8.51102889e-01, -2.65342355e-01,
            4.04596478e-02,  3.37684080e-02, -1.93432048e-02,
            5.03367111e-02,  2.34500766e-01, -6.15331471e-01,
            1.24603474e+00, -8.84704411e-01],
          [-1.26657712e+00,  7.47588277e-01, -4.42470089e-02,
            1.70719400e-01, -2.87955076e-01,  3.66688371e-02,
            0.00000000e+00, -1.16530709e-01, -2.19395429e-01,
            3.77322584e-02, -6.50452375e-02,  1.35849154e+00,
            1.38459951e-01,  7.15751529e-01,  5.99472761e-01,
           -2.64303982e-

In [31]:
bottleneck.summary()

Model: "efficientnet-b1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, None, None,  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, None, None, 3 864         input_11[0][0]                   
__________________________________________________________________________________________________
stem_gn (GroupNormalization)    (None, None, None, 3 64          stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, None, None, 3 0           stem_gn[0][0]                    
____________________________________________________________________________________